# Quickstart

This notebook provides a brief introduction to nested-pandas, including the motivation and basics for working with the data structure. For more in-depth descriptions, see the other tutorial notebooks.

## Installation

With a valid Python environment, nested-pandas and it's dependencies are easy to install using the `pip` package manager. The following command can be used to install it:

In [1]:
# % pip install nested-pandas

## Overview

Nested-Pandas is tailored towards efficient analysis of nested data sets. This includes data that would normally be represented in a Pandas DataFrames with multiple rows needed to represent a single "thing" and therefor multiple columns whose values will be identical for that item.

As a concrete example, consider an astronomical data set storing information about observations of physical objects, such as stars and galaxies. One way to represent this in Pandas is to create one row per observation with an ID column indicating to which physical object the observation corresponds. However this approach ends up repeating a lot of data over each observation of the same object such as its location on the sky (RA, dec), its classification, etc. Further any operations processing the data as time series requires the user to first perform a (potentially expensive) group-by operation.

In [ ]:
import pandas as pd

# Represent nested time series information as a classic pandas dataframe.
my_data_frame = pd.DataFrame(
    {
        "id": [0, 0, 0, 1, 1],
        "ra": [10.0, 10.0, 10.0, 15.0, 15.0],
        "dec": [0.0, 0.0, 0.0, -1.0, -1.0],
        "time": [60676.0, 60677.0, 60678.0, 60675.0, 60676.5],
        "brightness": [100.0, 101.0, 99.8, 5.0, 5.01],
    }
)
my_data_frame

Beyond astronomical data we might be interested in tracking a patients blood pressure over time, a measure of intensities at different wavelengths, or storing a list of the type of rock found at different depths of core samples. In each case it is possible to represent this data with multiple rows (one row for each patient + measurement pair) and associate them together by ids.

In contrast, nested-pandas allows columns to represent nested data. We can have columns with the (single) value for the objects’ unvarying characteristics (location on the sky, patentient birth date, location of the core sample) and nested columns for the values of each observation.

Let's see an example:

In [ ]:
from nested_pandas.datasets import generate_data

# generate_data creates some toy data
nf = generate_data(10, 100)  # 10 rows, 100 nested rows per row
nf

The above dataframe is a `NestedFrame`, which extends the capabilities of the Pandas `DataFrame` to support columns with nested information. 

In this example, we have the top level dataframe with 10 rows and 2 typical columns, "a" and "b".  These represent object level attributes. The "nested" column contains a dataframe in each row that represents a time series (or observation level values).  As we will see below, this allows easy access to the all of the observations for a given object.

## Accessing Nested Data

We can inspect the contents of the "nested" column using pandas API tooling like `loc`.

In [ ]:
nf.loc[0]["nested"]

Here we see that within the "nested" column there are `NestedFrame` objects with their own data. In this case we have 3 columns ("t", "flux", and "band") that represent a time series of observations. 

Note that `loc` itself accesses the row, so the combination of `nf.loc[0]["nested"]` means we are looking at value in the "nested" column for a single row (row 0). If we just use `nf.loc[0]` we would retrieve the entire row, including the nested colum and all other columns. Similarly if we use `nf["nested”]` we retrieve the nested column for all rows. What makes the nesting useful is that once we access the nested entry for a specific row, we can treat the value as a table in its own right.

As in Pandas, we can still access individual entries from a column ased on the row index. Thus we can access the entry (table) in row 0 of the nested column as `nf["nested"][0]` as well.

In [ ]:
nf["nested"][0]

We can also use dot notation to access all the values in a nested sub column:

In [ ]:
nf["nested.t"]

Note that "nested.t" contains the t values for all rows, but preserves the nesting information. The id column of the returned data maps the top-level row (in `nf`) whether this value resides.

Similarly we can access the values for a given top-level row by index. To get all the `t` values for row 0 we could specify:

In [ ]:
nf["nested.t"][0]

Here the `[0]` is telling our nested frame to access the values of the series `nf["nested.t"]` where the id = 0. If we try `nf["nested.t"][0][0]` we again match id = 0 and return the same frame. So to access a single element within the series, we can use its location:

In [ ]:
nf["nested.t"][0].iloc[0]

## Inspecting Nested Frames

We can inspect the available columns using some custom properties of the `NestedFrame`.

In [ ]:
# Shows which columns have nested data
nf.nested_columns

In [ ]:
# Provides a dictionary of "base" (top-level) and nested column labels
nf.all_columns

## Pandas Operations

Nested-pandas extends the Pandas API, meaning any operation you could do in Pandas is available within nested-pandas. However, nested-pandas has additional functionality and tooling to better support working with nested datasets. For example, let's look at `query`:

In [ ]:
# Normal queries work as expected, rejecting rows from the dataframe that don't meet the criteria
nf.query("a > 0.2")

The above query is native Pandas, however with nested-pandas we can use hierarchical column names to extend `query` to nested layers.

In [ ]:
# Applies the query to "nested", filtering based on "t >17"
nf_g = nf.query("nested.t > 17.0")
nf_g

This query does not affect the rows of the top-level dataframe, but rather applies the query to the "nested" dataframes. If we look at one of them, we can see the effect of the query.

In [ ]:
# All t <= 17.0 have been removed
nf_g.loc[0]["nested"]

A limited set of functions have been extended in this way so far, with the aim being to fully support this hierarchical access where applicable in the Pandas API.

## Reduce Function

Finally, we'll end with the flexible `reduce` function. `reduce` functions similarly to Pandas' `apply` but flattens (reduces) the inputs from nested layers into array inputs to the given apply function. For example, let's find the mean flux for each dataframe in "nested":

In [ ]:
import numpy as np

# use hierarchical column names to access the flux column
# passed as an array to np.mean
nf.reduce(np.mean, "nested.flux")

This can be used to apply any custom functions you need for your analysis, and just to illustrate that point further let's define a custom function that just returns it's inputs.

In [15]:
def show_inputs(*args):
    return args

Applying some inputs via reduce, we see how it sends inputs to a given function.

In [ ]:
nf_inputs = nf.reduce(show_inputs, "a", "nested.band")
nf_inputs

In [ ]:
nf_inputs.loc[0]